# CPS600 - Python Programming for Finance 
###  
<img src="https://www.syracuse.edu/wp-content/themes/g6-carbon/img/syracuse-university-seal.svg?ver=6.3.9" style="width: 200px;"/>

# Python IO

###  September 11, 2018

### From last time...

## File IO & Text Processing

### Now we're going to see some basic IO operations and learn how to process text by following an example from *Think Python, $2^{nd}$ Edition*.

## *Example - Processing Text from a Book*
>Write a program that reads a file, breaks each line into words, strips whitespace and punctuation from the words and converts them to lowercase.

### Here is one quick way to download a text from inside our Python session.

In [132]:
from urllib import request
request.urlretrieve ("https://www.gutenberg.org/files/158/158-0.txt", "Emma.txt")

('Emma.txt', <http.client.HTTPMessage at 0x7fbc7aa36358>)

### While we're at it, let's download the `words.txt` from *Think Python*.

In [133]:
fileURL = "https://raw.githubusercontent.com/AllenDowney/ThinkPython2/master/code/words.txt"
request.urlretrieve (fileURL, "words.txt")

('words.txt', <http.client.HTTPMessage at 0x7fbc7aa365f8>)

### Let's import `time` so that we can make a comparison.

In [42]:
import time

### Here are two different ways to write the desired function.

In [43]:
def make_word_list1():
    """Reads lines from a file and builds a list using append."""
    t = []
    fin = open('words.txt')
    for line in fin:
        word = line.strip()
        t.append(word)
    return t


def make_word_list2():
    """Reads lines from a file and builds a list using list +."""
    t = []
    fin = open('words.txt')
    for line in fin:
        word = line.strip()
        t = t + [word]
    return t


### Below, we compare these two functions. Which one is the faster of the two?

In [ ]:
start_time = time.time()
t = make_word_list1()
elapsed_time = time.time() - start_time

print(len(t))
print(t[:10])
print(elapsed_time, 'seconds')

start_time = time.time()
t = make_word_list2()
elapsed_time = time.time() - start_time

print(len(t))
print(t[:10])
print(elapsed_time, 'seconds')

### Next, we want to clean up the book and compute frequency statistics - what are the words in the book, and how many times is each one used?

In [45]:
import string # Used to get punctuation

In [89]:
def process_file(filename, skip_header):
    """Makes a histogram that contains the words from a file.

    filename: string
    skip_header: boolean, whether to skip the Gutenberg header
   
    returns: map from each word to the number of times it appears.
    """
    hist = {} # This is an empty dictionary
    fp = open(filename)

    if skip_header:
        skip_gutenberg_header(fp)

    for line in fp:
        process_line(line, hist)

    return hist


def skip_gutenberg_header(fp):
    """Reads from fp until it finds the line that ends the header.
    
    RMK: You just have to look at the Gutenberg format. That is
    how you would know how to write such a function. This had to
    be changed.

    fp: open file object
    """
    for line in fp:
        if line.startswith('*** START OF THIS PROJECT GUTENBERG EBOOK EMMA ***'):
            break


def process_line(line, hist):
    """Adds the words in the line to the histogram.

    Modifies hist.
    
    RMK: This is not *pure* function. It modifies
    one of its arguments. This is frowned upon
    in many circles, but it is one way to do things.

    line: string
    hist: histogram (map from word to frequency)
    """
    # replace hyphens with spaces before splitting
    line = line.replace('-', ' ')
    strippables = string.punctuation + string.whitespace

    for word in line.split():
        # remove punctuation and convert to lowercase
        word = word.strip(strippables)
        word = word.lower()

        # update the histogram
        hist[word] = hist.get(word, 0) + 1



### We want to compute word statistics for a document.

### The `hist` dictionary object contains all the information about our word stats. It is easy to write functions that compute word count and *unique* word count.

In [55]:
def total_words(hist):
    """Returns the total of the frequencies in a histogram."""
    return sum(hist.values())


def different_words(hist):
    """Returns the number of different words in a histogram."""
    return len(hist)

### Finally, our functions for the most commonly occurring words.

In [56]:
def most_common(hist):
    """Makes a list of word-freq pairs in descending order of frequency.

    hist: map from word to frequency

    returns: list of (frequency, word) pairs
    """
    t = []
    for key, value in hist.items():
        t.append((value, key))

    t.sort()
    t.reverse()
    return t


def print_most_common(hist, num=10):
    """Prints the most commons words in a histgram and their frequencies.
    
    hist: histogram (map from word to frequency)
    num: number of words to print
    """
    t = most_common(hist)
    print('The most common words are:')
    for freq, word in t[:num]:
        print(word, '\t', freq)


### OK, now it is time to use all of our functions. Let's try it on *Emma* first.

In [ ]:
hist = process_file('Emma.txt', skip_header=True)
print('Total number of words:', total_words(hist))
print('Number of different words:', different_words(hist))

t = most_common(hist)
print('The most common words are:')
for freq, word in t[0:20]:
    print(word, '\t', freq)

### More File IO

In [6]:
import json

### The `json` module gives us one way to work with dictionary objects, storing and loading them.

In [11]:
ourRoom = {'desks':12, 'screens':2, 'people': 20}

In [12]:
with open('ourRoom.json','w') as f:
    json.dump(ourRoom,f)

In [13]:
with open('ourRoom.json','r') as f:
    newRoom = json.load(f)

In [14]:
newRoom

{'desks': 12, 'screens': 2, 'people': 20}

In [15]:
type(newRoom)

dict

## CSV Files

### An easy way to get CSV files in and out of an IPython session is to use `pandas` DataFrames.

In [16]:
import pandas as pd

In [ ]:
census = pd.read_csv('2010_Census_Populations_by_Zip_Code.csv')
census.head(4)

In [25]:
census['computedAHZ'] = census['Total Population'] / census['Total Households']

In [ ]:
census['computedAHZ'] - census['Average Household Size']

### We can then write the file back to the csv...

In [28]:
census.to_csv('2010_Census_Populations_by_Zip_Code.csv', index=False)

### And then read it back in, in order to check that our changes are reflected in the file.

In [ ]:
newCensus = pd.read_csv('2010_Census_Populations_by_Zip_Code.csv')
newCensus.head(3)

### Indeed. Speaking of `pandas` and `json`, DataFrame can also be read from `json` files, a common format for responses to API calls (such as the one we saw in lab last time).

In [31]:
newCensus.to_json('2010json.json')

In [ ]:
jsonCensus = pd.read_json('2010json.json')
jsonCensus.head(3)

In [ ]:
jsonCensus

### So, even though we think of `json` format as dictionary-like, `pandas` can (sometimes) read it as a DataFrame object. Here is that construction a bit more explicitly:

In [35]:
demoDict = {'col1':[1,2,3], 'col2':[.4,.45,0], 'col3':[0,0,0]}
demoFrame = pd.DataFrame(demoDict)

In [ ]:
demoFrame

# (Tuesday, Week 3)

### Let's take a tour of IO options for python. The thing you're most likely to use from this in the 'real world' is some interface to a SQL database, but there are others you should be aware of.

### Here, in broad strokes, are the main points to be considered:
1. Is your data stored in RAM? How much RAM, or *memory* is available on your system?
2. Is your data stored on disk? How fast can data be transferred to/from disk?
3. What is the file format, and how does it influence IO rate?

### Here is a depiction of it

<img src="ioflow.png" style="width: 400px;"/>

### Let's see our options.

## Basic IO with Python

### Observe first that you can move yourself around:

In [1]:
import os
os.chdir('..')

### I just moved 'up' a directory ( as I showed you in the terminal last week ). Let's look around...

In [ ]:
os.listdir()

### OK, what's in `data`?

In [ ]:
os.listdir('data')

### Let's load the *Old Faithful* data

In [2]:
import pandas as pd

In [9]:
olf = pd.read_csv('data/legit_olf.csv')

In [ ]:
olf.head()

### Let's go back home...

In [3]:
os.chdir('notebooks/')

### And look around...

In [ ]:
os.listdir()

### That was cheating a bit because we used `pandas`. Let's use the native Python capabilities for IO now.

In [4]:
import numpy as np
from random import gauss
a = [gauss(1.5,2) for i in range(1000000)]

### We will write the list `a` to disk:

In [14]:
import pickle
with open('list.pkl','wb') as pklFile: # You need the 'b' in that second argument
    %time pickle.dump(a,pklFile)

CPU times: user 33.6 ms, sys: 4.16 ms, total: 37.7 ms
Wall time: 36.6 ms


### In our text, you will find a different approach where the name `pklFile` is bound to the file object, then explicitly closed at the end of the operation. Here, you can see that one effect of the `with...as` construct is that the file is closed when you've finished:

In [ ]:
pklFile.closed

### Note another command appearing in the text: `ll`. It is another way to list the contents of the directory, and its default output has a bit more detail:

In [15]:
ll

total 9048
-rw-rw-r-- 1 martin    4593 Sep 10 20:23 Ch02_Infrastructure_Tools.ipynb
-rw-rw-r-- 1 martin   51571 Sep 10 20:22 Ch04_Data_Structures.ipynb
-rw-rw-r-- 1 martin   61313 Sep 11 06:42 ioflow.png
-rw-rw-r-- 1 martin 9002006 Sep 11 11:21 list.pkl
-rw-rw-r-- 1 martin  119996 Sep 10 20:24 PyFiWeek2.ipynb
-rw-rw-r-- 1 martin    8364 Sep 11 11:21 PyIO.ipynb
-rw-rw-r-- 1 martin    1273 Sep 10 20:28 PyObj.ipynb


### Let's read our file back in.

In [16]:
with open('list.pkl','rb') as pklFile: # You need the 'b' in that second argument
    b = pickle.load(pklFile)

In [ ]:
type(b)

### Let's confirm that these two are the same.

In [ ]:
a == b

### That worked. But generally you can use this handy `numpy` function:

In [ ]:
np.allclose(a,b) # Tolerates inequality below some threshold

### Now, multiple objects:

In [25]:
with open('list.pkl','wb') as pklFile:
    %time pickle.dump(np.array(a), pklFile)
    %time pickle.dump(np.array(a) ** 2, pklFile)    

CPU times: user 34.5 ms, sys: 11.9 ms, total: 46.5 ms
Wall time: 46.4 ms
CPU times: user 39.1 ms, sys: 4.37 ms, total: 43.4 ms
Wall time: 33.5 ms


In [26]:
ll

total 15884
-rw-rw-r-- 1 martin     4593 Sep 10 20:23 Ch02_Infrastructure_Tools.ipynb
-rw-rw-r-- 1 martin    51571 Sep 10 20:22 Ch04_Data_Structures.ipynb
-rw-rw-r-- 1 martin    61313 Sep 11 06:42 ioflow.png
-rw-rw-r-- 1 martin 16000322 Sep 11 11:26 list.pkl
-rw-rw-r-- 1 martin   119996 Sep 10 20:24 PyFiWeek2.ipynb
-rw-rw-r-- 1 martin    10227 Sep 11 11:25 PyIO.ipynb
-rw-rw-r-- 1 martin     1273 Sep 10 20:28 PyObj.ipynb


### Note the the file is double the size it was before. In your text it is remarked that this operation was faster with the use of `np.array`, but that no longer seems to be true.

### The objects have been *written to disk*, whereas the `a` and `b` should be thought of as object sitting in RAM.

### Let's load those two values back in...

In [29]:
with open ('list.pkl','rb') as pklFile:
    x = pickle.load(pklFile)

In [ ]:
x[:5]

### What gives? Let's try something else...

In [31]:
with open ('list.pkl','rb') as pklFile:
    x = pickle.load(pklFile)
    y = pickle.load(pklFile)

In [ ]:
x[:5], y[:5]

### Let's compare...

In [ ]:
np.array(a[:5]), np.array(a[:5])**2

### Looking good, but that was inconvenient. So let's store `x,y` in a dictionary object, and then write that object to our file using `pickle`.

In [37]:
with open('list.pkl', 'wb') as pklFile:
    pickle.dump({'x':x,'y':y}, pklFile)

### Now, when we load it in and find that it is an dictionary object.

In [ ]:
with open('list.pkl','rb') as pklFile:
    data = pickle.load(pklFile)
for key in data.keys():
    print( key, data[key][:5])

## Text Files, or CSV

### Here, we'll do it 'manually', starting with some `pandas` date range data.

In [41]:
rows = 5000
a = np.random.standard_normal((rows,5))
t = pd.date_range(start='2014/1/1', periods=rows, freq='H')

In [ ]:
a,t

### Here we are writing it to a `csv` file manually (in the way that we have done so far):

In [42]:
"{0:s},{1:f}".format('ape',5)

'ape,5.000000'

### Look at that `b` in the second string argument to our writing method. That puts the file object in binary mode. This used to be an issue just for Windows users, but as of *Python3*, everyone has to worry about it.

In [52]:
with open('data.csv','wb') as csvFile:
    header = b'data,no1,no2,no3,no4,no5\n'
    csvFile.write(header)
    for t_, (no1,no2,no3,no4,no5) in zip(t,a):
        s = bytes("{0:s},{1:f}{2:f}{3:f}{4:f}{5:f}".format(t_,no1,no2,no3,no4,no5), 'utf-8')
        csvFile.write(s)


### *Question for You All*: why couldn't I just prepend that string inside the loop with a `b` as well? What happens if you do?

### Before moving on from this painful way of doing IO, let's read that guy back in:

In [ ]:
with open('data.csv','r') as csvFile:
    for i in range(5):
        print(csvFile.readline())

### This really makes you appreciate pandas. More on that in a few...

# SQL

In [58]:
import sqlite3 as sq3

In [60]:
query = 'CREATE TABLE numbs (Date date,No1 real,No2 real)'
con = sq3.connect('numbs.db') # Creating a SQL connection object

### Note how we have broken this down into:
1. A query string - imagine contexts where you might programmatically format such a string.
2. A connection object.

### Now we're going to execute that query:

In [ ]:
con.execute(query)

In [63]:
con.commit() # This must be done in order to make the change effective.

### OK....so what? So look at the `query` string - it's actually not much of a query, but in fact tells SQL to *create a table*. So that's what it did for us.

##### (For fun, try running that same 'query' again)

### This will actually put some data into our `numbs` table:

In [64]:
import datetime as dt
con.execute('INSERT INTO numbs VALUES (?,?,?)',
           (dt.datetime.now(), 0.12, 4.4))

### We can do the same thing inside of a loop in order to write multiple rows:

In [ ]:
data = np.random.standard_normal((10000,2)).round(5)

for row in data:
    con.execute('INSERT INTO numbs VALUES (?,?,?)',
                (dt.datetime.now(), row[0],row[1]))
con.commit() # NB, this is outside of the loop.

### Does anyone else feel like we're flying blind? Let's look at what we've got...

In [ ]:
con.execute('SELECT * from numbs').fetchmany(10)

### Bretty gool! Takeaway:
    - Python is already set up to interact with existing (legacy) data frameworks
    - You are freed up to deal with just the Python objects - and the occasional piece of SQL syntax

## Pandas Again

### Pandas supports many file formats. Let's look at a few and compare performance (in terms of IO speed).

### We begin by generating a dummy dataset for use in our comparisons.

In [68]:
data = np.random.standard_normal((1000000, 5)).round(5)
filename = 'numbs' # We'll re-use this, so it's handy to name it.

### We just saw SQL above. Let's use it to establish a baseline.

In [ ]:
query = 'CREATE TABLE numbers (No1 real, No2 real, No3 real, No4 real, No5 real)'
con = sq3.Connection(filename+'.db')
con.execute(query)

### Here, we'll insert our data and time how long it takes:

In [ ]:
%%time
con.executemany('INSERT INTO numbers VALUES (?,?,?,?,?)', data)
con.commit()

### How about reading it?

In [ ]:
%%time
temp = con.execute('SELECT * FROM numbers').fetchall()
print(temp[:2])
temp = 0.0

### Much faster than writing. We can alternatively read results directly into a `numpy` array.

In [ ]:
%%time
query = 'SELECT * FROM numbers WHERE No1 > 0 AND No2 < 0'
res = np.array(con.execute(query).fetchall()).round(3)

### Can you picture that in your head? No need, we can have a look:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
res = res[::100] # Every hundredth datapoint
plt.plot(res[:,0],res[:,1],'ro')
plt.grid(True)
plt.xlim(-.5,4.5)
plt.ylim(-4.5,.5)

### In the examples above, we were using SQL. Let's use `pandas` to load a SQL table into *RAM* instead, dealing with it as a Python object *in memory*.

In [76]:
import pandas.io.sql as pds

In [ ]:
%time data = pds.read_sql('SELECT * FROM numbers',con)

### (*RMK* here and previously a *wildcard* symbol \* is used.)

### Let's have a look:

In [ ]:
data.head

### Let's do the equivalent query on our *DataFrame* object:

In [ ]:
data[(data['No1'] > 0) & (data['No2'] < 0)].head()

### We have already seen how to deal with `csv` files. Note also that we can use *Excel* files.

In [ ]:
%time data[:100000].to_excel(filename + '.xlsx')

In [ ]:
%time data[:100000].to_csv(filename + '.csv')

### (But it takes longer)

## PyTables

In [85]:
import tables as tb

In [87]:
filename = 'tab.h5' # New filename for HDF5 format

### Opening the file...

In [88]:
h5 = tb.open_file(filename, 'w')

In [90]:
rows = 2000000 # The number of rows

### Let's describe the table we desire: it has a datetime column, two int and two float columns.

In [92]:
row_des = {
'Date': tb.StringCol(26, pos=1),
'No1': tb.IntCol(pos=2),
'No2': tb.IntCol(pos=3),
'No3': tb.Float64Col(pos=4),
'No4': tb.Float64Col(pos=5)
}

In [95]:
filters = tb.Filters(complevel=0) # no compression

tab = h5.create_table('/', 'ints_floats', row_des,
    title = 'Integers and Floats',
    expectedrows = rows, filters = filters)

In [ ]:
tab

In [99]:
pointer = tab.row

### Generating some data:

In [101]:
ran_int = np.random.randint(0,10000,size=(rows,2))
ran_flo = np.random.standard_normal((rows,2)).round(5)

### Finally, let's write our data.

In [102]:
%%time
for i in range(rows):
    pointer['Date'] = dt.datetime.now()
    pointer['No1'] = ran_int[i,0]
    pointer['No2'] = ran_int[i,1]
    pointer['No3'] = ran_flo[i,0]
    pointer['No4'] = ran_flo[i,1]
    pointer.append() # Advances the pointer.
tab.flush() # Like 'commit' in the SQL examples

CPU times: user 5.81 s, sys: 91.6 ms, total: 5.91 s
Wall time: 5.9 s


In [103]:
tab

/ints_floats (Table(2000000,)) 'Integers and Floats'
  description := {
  "Date": StringCol(itemsize=26, shape=(), dflt=b'', pos=0),
  "No1": Int32Col(shape=(), dflt=0, pos=1),
  "No2": Int32Col(shape=(), dflt=0, pos=2),
  "No3": Float64Col(shape=(), dflt=0.0, pos=3),
  "No4": Float64Col(shape=(), dflt=0.0, pos=4)}
  byteorder := 'little'
  chunkshape := (2621,)